In [ ]:
pip install Mastodon.py

In [ ]:
pip install pytube

In [ ]:
pip install beautifulsoup4

In [9]:
import mastodon
from mastodon import Mastodon
from pytube import YouTube
from bs4 import BeautifulSoup
import re
import requests
import urllib


# Log in to Mastodon

In [ ]:
# creates an app for mastodon social
Mastodon.create_app(
    'hack-posttoots',
    api_base_url = 'https://mastodon.social',
    scopes=['read', 'write'],
    to_file = '.secrets'
)

In [11]:
# saves secrets
mastodon = Mastodon (
    client_id = '.secrets',
)

In [12]:
# requires an existing login

username = '<EMAIL>'
password = '<PASSWORD>'

mastodon.access_token = mastodon.log_in (
  username = username,
  password = password,
  scopes = ['read', 'write']
)
account = mastodon.me()

In [34]:
# some methods for posting and grabbing toot posted on your timeline

# post_toot = mastodon.status_post('Tooting from the command line!')
# toot_status = mastodon.status(post_toot.id)
# mastodon.status_delete(id)
# statuses = mastodon.account_statuses(id=account.id)

# Mastodon parsing

In [15]:
# helpers to parse mastodon payload

def youtube_url_validation(url):
    youtube_regex = (
        r'(https?://)?(www\.)?'
        '(youtube|youtu|youtube-nocookie)\.(com|be)/'
        '(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})')

    youtube_regex_match = re.match(youtube_regex, url)
    if youtube_regex_match:
        return youtube_regex_match

    return youtube_regex_match

def image_validation(filename):
  image_regex = (r'(.*)(\w+)(.gif|.jpg|.jpeg|.tiff|.png)')
  image_regex_match = re.match(image_regex, filename)
  if image_regex_match:
    return image_regex_match
  return image_regex_match

def grab_image_link(link):
  if image_validation(link):
    return link
  return None

def grab_video_link(link):
  if youtube_url_validation(link):
    return link
  return None

def grab_links(status_obj):
  image_urls = []
  video_urls = []
  for media_attachment in status_obj['media_attachments']:
    p_image_url = grab_image_link(media_attachment['url'])
    if p_image_url:
      image_urls.append(p_image_url)

  soup = BeautifulSoup(status_obj['content'])
  for a_tag in soup.findAll('a'):
    p_video_url = grab_video_link(a_tag.get('href'))
    if p_video_url:
      video_urls.append(p_video_url)
  url_dict = {
      'image_urls': image_urls,
      'video_urls': video_urls
  }
  return url_dict

def download_image(link, image_name):
  img_data = requests.get(link).content
  with open(f'./{image_name}.jpg', 'wb') as handler:
      handler.write(img_data)
  return

In [20]:
# grab media only from
public_statuses = mastodon.timeline_public(only_media=True)

In [ ]:
for ps in public_statuses:
  print(id, grab_links(ps))